In [5]:
import logging
import grpc
import time
import numpy as np
import pandas as pd
import server_tools_pb2
import server_tools_pb2_grpc
from google.protobuf import empty_pb2
from matplotlib import pyplot as plt

PORT = '50051'

def run_facile(channel, data_send, num_data_send, batch_size):
    start_time = time.time()
    # Get a client ID which you need to talk to the server
    stub = server_tools_pb2_grpc.FacileServerStub(channel)
    try:
        #logging.info("Request client id from server")
        response = stub.RequestClientID(server_tools_pb2.NullParam())
    except BaseException:
        print(
             """Connection to the server could not be established.
             Press enter to try again.""")
        return
    client_id = response.new_id
    #logging.info("Client id is " + str(client_id))
    finish_time = time.time()-start_time

    #print("Time establishing server connections ", finish_time)
    # Send the data to the server and receive an answer
    start_time = time.time()
    #logging.info("Number tested is " + str(num_data_send))
    #logging.info("Submitting files and waiting")
    data_message = server_tools_pb2.DataMessage(
        client_id=client_id, data=data_send, batch_size=batch_size)
    response = stub.StartJobWait(data_message, 1000, [])

    # Print output
    whole_time = time.time() - start_time
    #print("Whole time:", whole_time)
    #print("Predict time:", response.infer_time)
    #print("Fraction of time spent not predicting:",
    #      (1 - response.infer_time / whole_time) * 100, '%')
    A = np.frombuffer(response.prediction,dtype = np.float32)
    print(list(np.frombuffer(response.prediction,dtype = np.float32))[:10])
    infer_time_GPU = response.infer_time_GPU
    infer_time_CPU = response.infer_time_CPU
    channel.close()
    return infer_time_GPU, infer_time_CPU


def setup_server(host_IP):
    start_time = time.time()
    options = [('grpc.max_receive_message_length', 500*1024*1024 )]
    channel = grpc.insecure_channel(host_IP + ':' + PORT, options = options)
    
    finish_time = time.time()-start_time
    #print("Time defining server stub is ", finish_time)
    return channel

In [6]:
logging.basicConfig()
logging.root.setLevel(logging.NOTSET)
logging.basicConfig(level=logging.NOTSET)

start_time = time.time()
read_rec_hit = pd.read_pickle("input/X_HB.pkl")
read_rec_hit.drop(['PU', 'pt'], 1, inplace=True)
mu,std = np.mean(read_rec_hit, axis=0), np.std(read_rec_hit, axis=0)
print("mu is ", mu)
print("std is ", std)
read_rec_hit = (read_rec_hit-mu)/std

read_rec_hit = pd.concat([read_rec_hit]*100, ignore_index=True)


mu is  depth        2.243405
ieta         0.019543
iphi        36.519157
gain         0.000566
raw[0]     136.718005
raw[1]     163.566998
raw[2]     168.755297
raw[3]    4950.089370
raw[4]    2451.871581
raw[5]     808.803050
raw[6]     396.590292
raw[7]     266.167670
dtype: float64
std is  depth         1.045898
ieta          9.394450
iphi         20.708740
gain          0.000184
raw[0]      279.127180
raw[1]      331.065586
raw[2]      335.563759
raw[3]    14553.435515
raw[4]     6339.333508
raw[5]     1654.968567
raw[6]      644.929979
raw[7]      382.568971
dtype: float64


In [7]:
start_time = time.time()
read_rec_hit = pd.read_pickle("input/X_HB.pkl")
read_rec_hit.drop(['PU', 'pt'], 1, inplace=True)
mu,std = np.mean(read_rec_hit, axis=0), np.std(read_rec_hit, axis=0)
print("mu is ", mu)
print("std is ", std)
read_rec_hit = (read_rec_hit-mu)/std
print("Time used per rec hit", (time.time()-start_time)/len(read_rec_hit))

mu is  depth        2.243405
ieta         0.019543
iphi        36.519157
gain         0.000566
raw[0]     136.718005
raw[1]     163.566998
raw[2]     168.755297
raw[3]    4950.089370
raw[4]    2451.871581
raw[5]     808.803050
raw[6]     396.590292
raw[7]     266.167670
dtype: float64
std is  depth         1.045898
ieta          9.394450
iphi         20.708740
gain          0.000184
raw[0]      279.127180
raw[1]      331.065586
raw[2]      335.563759
raw[3]    14553.435515
raw[4]     6339.333508
raw[5]     1654.968567
raw[6]      644.929979
raw[7]      382.568971
dtype: float64
Time used per rec hit 1.5872557504557774e-06


In [8]:
from keras.models import load_model
start_time =time.time()
weight_file = load_model("weights.h5", compile=False)
finish_time = time.time() - start_time
print("Finish time", (time.time()-start_time)/len(compressed_data))

Using TensorFlow backend.
Level 1:tensorflow:Registering FakeQuantWithMinMaxArgs (<function _FakeQuantWithMinMaxArgsGradient at 0x7f6e971fcd08>) in gradient.
Level 1:tensorflow:Registering FakeQuantWithMinMaxVars (<function _FakeQuantWithMinMaxVarsGradient at 0x7f6e97200048>) in gradient.
Level 1:tensorflow:Registering FakeQuantWithMinMaxVarsPerChannel (<function _FakeQuantWithMinMaxVarsPerChannelGradient at 0x7f6e971fcea0>) in gradient.
Level 1:tensorflow:Registering MatMul,flops (<function _calc_mat_mul_flops at 0x7f6e9704e488>) in statistical functions.
Level 1:tensorflow:Registering AccumulateNV2 (<function _accumulate_n_grad at 0x7f6e9704ea60>) in gradient.
Level 1:tensorflow:Registering TensorListConcatLists (None) in gradient.
Level 1:tensorflow:Registering TensorListElementShape (None) in gradient.
Level 1:tensorflow:Registering TensorListLength (None) in gradient.
Level 1:tensorflow:Registering TensorListPushBackBatch (None) in gradient.
Level 1:tensorflow:Registering TensorLi

Level 1:tensorflow:Registering MaxPool3DGradGrad (<function _MaxPool3DGradGradGrad at 0x7f6e96274598>) in gradient.
Level 1:tensorflow:Registering Softmax (<function _SoftmaxGrad at 0x7f6e962740d0>) in gradient.
Level 1:tensorflow:Registering LogSoftmax (<function _LogSoftmaxGrad at 0x7f6e962742f0>) in gradient.
Level 1:tensorflow:Registering BiasAdd (<function _BiasAddGrad at 0x7f6e96274840>) in gradient.
Level 1:tensorflow:Registering BiasAddGrad (<function _BiasAddGradGrad at 0x7f6e962749d8>) in gradient.
Level 1:tensorflow:Registering BiasAddV1 (<function _BiasAddGradV1 at 0x7f6e96274ae8>) in gradient.
Level 1:tensorflow:Registering Relu (<function _ReluGrad at 0x7f6e96274950>) in gradient.
Level 1:tensorflow:Registering EluGrad (<function _EluGradGrad at 0x7f6e96274c80>) in gradient.
Level 1:tensorflow:Registering SeluGrad (<function _SeluGradGrad at 0x7f6e96274bf8>) in gradient.
Level 1:tensorflow:Registering Relu6 (<function _Relu6Grad at 0x7f6e96274f28>) in gradient.
Level 1:te

Level 1:tensorflow:Registering MatrixSetDiag (<function _MatrixSetDiagGrad at 0x7f6e96123d90>) in gradient.
Level 1:tensorflow:Registering MatrixBandPart (<function _MatrixBandPartGrad at 0x7f6e96123ea0>) in gradient.
Level 1:tensorflow:Registering EditDistance (None) in gradient.
Level 1:tensorflow:Registering Fill (<function _FillGrad at 0x7f6e96123f28>) in gradient.
Level 1:tensorflow:Registering ZerosLike (None) in gradient.
Level 1:tensorflow:Registering OnesLike (None) in gradient.
Level 1:tensorflow:Registering PreventGradient (<function _PreventGradientGrad at 0x7f6e961262f0>) in gradient.
Level 1:tensorflow:Registering Gather (<function _GatherGrad at 0x7f6e96126400>) in gradient.
Level 1:tensorflow:Registering GatherV2 (<function _GatherV2Grad at 0x7f6e96126510>) in gradient.
Level 1:tensorflow:Registering GatherNd (<function _GatherNdGrad at 0x7f6e96126620>) in gradient.
Level 1:tensorflow:Registering CheckNumerics (<function _CheckNumericsGrad at 0x7f6e96126730>) in gradien

Level 1:tensorflow:Registering TensorSummary (None) in gradient.
Level 1:tensorflow:Registering TensorSummaryV2 (None) in gradient.
Level 1:tensorflow:Registering Timestamp (None) in gradient.
Level 1:tensorflow:Registering Roll (<function _RollGrad at 0x7f6e962746a8>) in gradient.
Level 1:tensorflow:Registering ArgMax (<function _ArgMaxGrad at 0x7f6e96115158>) in gradient.
Level 1:tensorflow:Registering ArgMin (<function _ArgMinGrad at 0x7f6e961152f0>) in gradient.
Level 1:tensorflow:Registering Sum (<function _SumGrad at 0x7f6e96279730>) in gradient.
Level 1:tensorflow:Registering Max (<function _MaxGrad at 0x7f6e96115268>) in gradient.
Level 1:tensorflow:Registering Min (<function _MinGrad at 0x7f6e96115730>) in gradient.
Level 1:tensorflow:Registering Mean (<function _MeanGrad at 0x7f6e961157b8>) in gradient.
Level 1:tensorflow:Registering Prod (<function _ProdGrad at 0x7f6e96115510>) in gradient.
Level 1:tensorflow:Registering SegmentSum (<function _SegmentSumGrad at 0x7f6e9611584

Level 1:tensorflow:Registering Greater (None) in gradient.
Level 1:tensorflow:Registering GreaterEqual (None) in gradient.
Level 1:tensorflow:Registering Equal (None) in gradient.
Level 1:tensorflow:Registering ApproximateEqual (None) in gradient.
Level 1:tensorflow:Registering NotEqual (None) in gradient.
Level 1:tensorflow:Registering LogicalAnd (None) in gradient.
Level 1:tensorflow:Registering LogicalOr (None) in gradient.
Level 1:tensorflow:Registering LogicalNot (None) in gradient.
Level 1:tensorflow:Registering Select (<function _SelectGrad at 0x7f6e960a80d0>) in gradient.
Level 1:tensorflow:Registering MatMul (<function _MatMulGrad at 0x7f6e960a89d8>) in gradient.
Level 1:tensorflow:Registering SparseMatMul (<function _SparseMatMulGrad at 0x7f6e960a8840>) in gradient.
Level 1:tensorflow:Registering Floor (<function _FloorGrad at 0x7f6e960a8598>) in gradient.
Level 1:tensorflow:Registering Ceil (<function _CeilGrad at 0x7f6e960a8d08>) in gradient.
Level 1:tensorflow:Registering 

Level 1:tensorflow:Registering TensorArraySplitV2 (<function _TensorArraySplitGrad at 0x7f6e95f1d620>) in gradient.
Level 1:tensorflow:Registering TensorArraySplit (<function _TensorArraySplitGrad at 0x7f6e95f1d620>) in gradient.
Level 1:tensorflow:Registering LookupTableFind (None) in gradient.
Level 1:tensorflow:Registering LookupTableFindV2 (None) in gradient.
Level 1:tensorflow:Registering LookupTableInsert (None) in gradient.
Level 1:tensorflow:Registering LookupTableInsertV2 (None) in gradient.
Level 1:tensorflow:Registering LookupTableSize (None) in gradient.
Level 1:tensorflow:Registering LookupTableSizeV2 (None) in gradient.
Level 1:tensorflow:Registering HashTable (None) in gradient.
Level 1:tensorflow:Registering HashTableV2 (None) in gradient.
Level 1:tensorflow:Registering InitializeTable (None) in gradient.
Level 1:tensorflow:Registering InitializeTableV2 (None) in gradient.
Level 1:tensorflow:Registering InitializeTableFromTextFile (None) in gradient.
Level 1:tensorflow:

Level 1:tensorflow:Registering AddN,flops (<function _add_n_flops at 0x7f6e2cc41488>) in statistical functions.
Level 1:tensorflow:Registering Fact (<function _set_call_cpp_shape_fn.<locals>.call_without_requiring at 0x7f6e9737a1e0>) in default shape functions.
Level 1:tensorflow:Registering SdcaFprint (None) in gradient.
Level 1:tensorflow:Registering SdcaOptimizer (None) in gradient.
Level 1:tensorflow:Registering SdcaOptimizerV2 (None) in gradient.
Level 1:tensorflow:Registering SdcaShrinkL1 (None) in gradient.
Level 1:tensorflow:Registering queue_runners ((<class 'tensorflow.core.protobuf.queue_runner_pb2.QueueRunnerDef'>, <function QueueRunner.to_proto at 0x7f6e2cbc6488>, <function QueueRunner.from_proto at 0x7f6e2cbc6510>)) in proto functions.
Level 1:tensorflow:Registering GenerateVocabRemapping (None) in gradient.
Level 1:tensorflow:Registering LoadAndRemapMatrix (None) in gradient.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updatin

NameError: name 'compressed_data' is not defined

In [9]:
import time
start_time = time.time()
compressed_data = read_rec_hit.to_json().encode('utf-8')
print("Time used in compression ", (time.time()-start_time)/len(compressed_data))
start_time =time.time()
X = pd.read_json(compressed_data.decode('utf-8')) # OK!
finish_time = time.time()-start_time
print("Time used in decoding", (time.time()-start_time)/len(X))

Time used in compression  8.019828866272123e-09
Finish time 1.845098228234296e-05


In [13]:
import time
start_time = time.time()
numpy_read_rec_hit = read_rec_hit.values
compressed_data = numpy_read_rec_hit.tostring()
print(numpy_read_rec_hit[:10])
print("Time used in compression ", (time.time()-start_time)/len(compressed_data))
start_time =time.time()
X = np.frombuffer(compressed_data).reshape((numpy_read_rec_hit.shape[0],numpy_read_rec_hit.shape[1])) # OK!
print(X[:10])
print("After compression Shape", X.shape)
finish_time = time.time()-start_time
print("Time used in decoding", (time.time()-start_time)/len(X))


[[-0.23272329 -1.2794302   0.2163745   0.61245934  0.2476451   0.18444246
   0.41844795 -0.32856991 -0.3083652  -0.30189805 -0.41228662 -0.25589965]
 [-0.23272329 -1.17298438  0.50610724  0.53779327  0.18043785 -0.09925469
  -0.11338565 -0.25561329 -0.26091846 -0.30186196 -0.3248526  -0.35408121]
 [-0.23272329 -1.2794302   0.50610724  0.61245934 -0.15576349 -0.32709933
   0.53863217  2.05382297  1.89916078  1.79851572  1.78126762  0.81323943]
 [ 0.72339269 -1.49232185  0.26466329  0.6090654  -0.22339    -0.26944272
  -0.22554276 -0.29324557 -0.2673282  -0.23465689 -0.13704163 -0.11015938]
 [-1.18883927 -1.59876768  0.26466329 -1.36163373 -0.1561235   0.12832907
   0.05506634  0.07811421  0.03906849  0.06220861 -0.13495215 -0.10794536]
 [-1.18883927 -1.59876768  0.31295208 -1.36163373  1.15873696  1.5780543
   0.75620644  0.07816567  0.13412621  0.08501617  0.39038961  0.40866645]
 [-0.23272329 -1.49232185  0.26466329  0.6090654  -0.29080264 -0.2127894
  -0.28138282 -0.31114363 -0.34837

In [ ]:
IP = "localhost"
num_send = 16000
read_rec_hit_sliced = read_rec_hit[:num_send]
print(len(read_rec_hit))
finish_time = time.time()-start_time
print("Time reading data from local file and preprocessing (pkl->pandas) is ", finish_time)

start_time = time.time()
compressed_data = read_rec_hit_sliced.to_json().encode('utf-8')
finish_time = time.time()-start_time
print("Time reading data from local file (pandas->bytes) is ", finish_time)

gpu_infer_time = []
cpu_infer_time = []
batch_size_exp = np.arange(8,16)
for i in 2**batch_size_exp:
    gpu_time, cpu_time = run_facile(setup_server(IP), compressed_data, num_send, i)
    gpu_infer_time.append(gpu_time)
    cpu_infer_time.append(cpu_time)

In [ ]:
N = 16000
mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)

import matplotlib.ticker as mtick

fig = plt.figure()

ax = fig.add_subplot(111)
ax.plot(np.array(batch_size_exp), np.array(gpu_infer_time)/num_send, 'o', label = 'gpu')
ax.plot(np.array(batch_size_exp), np.array(cpu_infer_time)/num_send, 'o', label = 'cpu')
ax.legend()
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
plt.title("Inference time (s) vs. log_2 batch size " + str(int(num_send/N)) + " event")
plt.show()

In [ ]:
IP = "localhost"
num_send = 160000
read_rec_hit_sliced = read_rec_hit[:num_send]
print(len(read_rec_hit))
finish_time = time.time()-start_time
print("Time reading data from local file and preprocessing (pkl->pandas) is ", finish_time)

start_time = time.time()
compressed_data = read_rec_hit_sliced.to_json().encode('utf-8')
finish_time = time.time()-start_time
print("Time reading data from local file (pandas->bytes) is ", finish_time)

gpu_infer_time = []
cpu_infer_time = []

batch_size_exp = np.arange(10,18)
for i in 2**batch_size_exp:
    gpu_time, cpu_time = run_facile(setup_server(IP), compressed_data, num_send, i)
    gpu_infer_time.append(gpu_time)
    cpu_infer_time.append(cpu_time)

In [ ]:
N = 16000
mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)

import matplotlib.ticker as mtick

fig = plt.figure()

ax = fig.add_subplot(111)
ax.plot(np.array(batch_size_exp), np.array(gpu_infer_time)/num_send, 'o', label = 'gpu')
ax.plot(np.array(batch_size_exp), np.array(cpu_infer_time)/num_send, 'o', label = 'cpu')
ax.legend()

ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
plt.title("Inference time (s) vs. log_2 batch size " + str(int(num_send/N)) + " event")
plt.show()

In [ ]:
IP = "localhost"
num_send = 640000
read_rec_hit_sliced = read_rec_hit[:num_send]
print(len(read_rec_hit))
finish_time = time.time()-start_time
print("Time reading data from local file and preprocessing (pkl->pandas) is ", finish_time)

start_time = time.time()
compressed_data = read_rec_hit_sliced.to_json().encode('utf-8')
finish_time = time.time()-start_time
print("Time reading data from local file (pandas->bytes) is ", finish_time)

gpu_infer_time = []
cpu_infer_time = []


batch_size_exp = np.arange(10,25)
for i in 2**batch_size_exp:
    gpu_time, cpu_time = run_facile(setup_server(IP), compressed_data, num_send, i)
    gpu_infer_time.append(gpu_time)
    cpu_infer_time.append(cpu_time)
print(gpu_infer_time)
print(cpu_infer_time)

In [ ]:
N = 16000
mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)

import matplotlib.ticker as mtick

fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(np.array(batch_size_exp), np.array(gpu_infer_time)/num_send, 'o', label = 'gpu')
ax.plot(np.array(batch_size_exp), np.array(cpu_infer_time)/num_send, 'o', label = 'cpu')
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
ax.legend()
plt.title("Inference time (s) vs. log_2 batch size " + str(int(num_send/N)) + " event")
plt.show()

In [ ]:
IP = "localhost"
num_send = 1600000
read_rec_hit_sliced = read_rec_hit[:num_send]
print(len(read_rec_hit))
finish_time = time.time()-start_time
print("Time reading data from local file and preprocessing (pkl->pandas) is ", finish_time)

start_time = time.time()
compressed_data = read_rec_hit_sliced.to_json().encode('utf-8')
finish_time = time.time()-start_time
print("Time reading data from local file (pandas->bytes) is ", finish_time)

gpu_infer_time = []
cpu_infer_time = []


batch_size_exp = np.arange(10,25)
for i in 2**batch_size_exp:
    gpu_time, cpu_time = run_facile(setup_server(IP), compressed_data, num_send, i)
    gpu_infer_time.append(gpu_time)
    cpu_infer_time.append(cpu_time)
print(gpu_infer_time)
print(cpu_infer_time)

In [ ]:
N = 16000
mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)

import matplotlib.ticker as mtick

fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(np.array(batch_size_exp), np.array(gpu_infer_time)/num_send, 'o', label = 'gpu')
ax.plot(np.array(batch_size_exp), np.array(cpu_infer_time)/num_send, 'o', label = 'cpu')
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
ax.legend()
plt.title("Inference time (s) vs. log_2 batch size " + str(int(num_send/N)) + " event")
plt.show()